# Import trajectories data spreadsheet

In [1]:
import tkinter.filedialog
import tkinter as tk
import os
root = tk.Tk() # open tkinter gui
rootDirectory = tk.filedialog.askdirectory() #browse for folder
root.destroy() # close tkinter
os.chdir(rootDirectory)

In [2]:
import tkinter.filedialog
import tkinter as tk
import pandas as pd

root = tk.Tk() # open tkinter gui
filepath = tk.filedialog.askopenfilename() # browse for file
root.destroy() # close tkinter
print(filepath)
df = pd.read_csv(filepath,header=0,sep=',',engine='python')
f = df.apply(pd.to_numeric,errors='coerce')

C:/Users/vanja/Box/Experimental/2020/Lawnmower_310120/Trajs with overlaps/Displacements/dRcum_LM_experiment1_hor&ver.csv


# Choose trajectories for analysis. Don't forget to press 'Enter' after you are done

In [40]:
import numpy as np
import re

chosenTracksInput = input("Please enter trajectory numbers separated by a comma, ranges, or type 'all'. Press 'Enter': ");
chosenTracksInput = chosenTracksInput.replace(" ", ""); #remove whitespaces from user's input
chosenTracksList = chosenTracksInput.split(","); #split input to list, comma is separator
allChosenTracksList = [];
#print(chosenTracksList)
for i in range (0,len(chosenTracksList)): #loop for chosen track numbers
    flag = False; #flag becomes true when input is correct
    chosenTrackNum = chosenTracksList[i];
    if chosenTrackNum.isdigit() == True:
        allChosenTracksList.append(chosenTrackNum);
        #print(allChosenTracksList)
        flag = True;
        print("You chose trajectory # " + chosenTrackNum)
        dfChosenTrack = df.loc[:,['X'+chosenTrackNum, 'Y'+chosenTrackNum, 'Flag'+chosenTrackNum]]; # dataframe for every chosen track
        if i == 0: #create dataframe if that't 1st track in input
            tracksDf = dfChosenTrack;
        else:    
            tracksDf = pd.concat([tracksDf, dfChosenTrack], axis=1); #otherwise add track to analysis dataframe
    elif chosenTrackNum == 'all': # if all tracks are chosen, they're added to analysis dataframe
        flag = True;
        print("You chose all trajectories")
        tracksDf = df;
        colnames = str(tracksDf.columns)
        s = [float(s) for s in re.findall(r'-?\d+\.?\d*', colnames)]
        allChosenTracksList = np.unique(np.array(s))
    elif (chosenTrackNum.find('-') != -1): #split dash-separeted ranges of tracks so to get all tracks within range
        tracksRange = chosenTrackNum.split("-");
        #print(tracksRange)
        if len(tracksRange) == 2 and  all(j.isdigit() == True for j in tracksRange): #check there're 2 numbers in input range
            flag = True
            allTracksInRange = np.arange(int(tracksRange[0]),int(tracksRange[1])+1); #make array of all numbers within range
            for t in allTracksInRange:
                chosenTrackNum = str(t);
                allChosenTracksList.append(chosenTrackNum);
                print("You chose trajectory # " + chosenTrackNum)
                dfChosenTrack = df.loc[:,['X'+chosenTrackNum, 'Y'+chosenTrackNum, 'Flag'+chosenTrackNum]];
                if i == 0 and t == allTracksInRange[0]:
                    tracksDf = dfChosenTrack; #create dataframe if that't 1st track in input
                else: 
                    tracksDf = pd.concat([tracksDf, dfChosenTrack], axis=1); #add other chosen tracks to analysis dataframe
    print("You chose in total " + str(len(allChosenTracksList)) + " trajectories")
    if flag == False: 
        print("Trajectories not found. Please re-enter trajectory number or 'all'")
        break

print(tracksDf) 
#tracksDf.to_csv('TracksForAnalysis.csv', index=False)

Note that TA MSDs calculation is time-intense – for a quicker result choose not more than 2 trajectories when running the notebook.
Please enter trajectory numbers separated by a comma, ranges, or type 'all'. Press 'Enter': all
You chose all trajectories
You chose in total 21 trajectories
      Time(s)          X1         Y1  Flag1         X3         Y3  Flag3  \
0           0  128.080747  21.007390    NaN  23.287917  25.057084    NaN   
1           1  128.111582  21.067830    NaN  23.967673  25.390085    NaN   
2           2  128.086236  21.270652    NaN  23.682352  24.818750    NaN   
3           3  128.231447  21.506928    NaN  23.258021  24.924441    NaN   
4           4  128.120190  21.150962    NaN  23.862501  25.287500    NaN   
...       ...         ...        ...    ...        ...        ...    ...   
6784     6784         NaN        NaN    NaN        NaN        NaN    NaN   
6785     6785         NaN        NaN    NaN        NaN        NaN    NaN   
6786     6786         NaN 

# Setting FPS and pixel size

In [6]:
fps = 1
pixSize = 1 # pixel size in um/pix

# Create dataframe

In [55]:
listOfFrames = []; #create empty list
for i in range (0, tracksDf.shape[0]): # loop for rows
    if np.isnan(df.iloc[i,0]) == False: # if frame exists in initial data, add zeros row for it in list
        listOfTracks = [df.iloc[i,0]];
        for j in range (1, int(tracksDf.shape[1] / 3) + 1): #Change denominator to how many columns each track occupies
            listOfTracks.extend(['NaN', 'NaN']); #grow list horizontally to encompass all tracks
        listOfFrames.append(listOfTracks); #grow list vertically to encompass all frames rows
    #else: 
        #break    
listOfColNames = ['Time(s)'];
for trackNum in allChosenTracksList:
    listOfColNames.extend(['d_x track ' + str(int(trackNum)), 'd_y track ' + str(int(trackNum))]);
dfdR = pd.DataFrame(listOfFrames, columns = listOfColNames) #make MSD dataframe from list
print("Empty dataframe for MSD was created")
print(dfdR)
print (tracksDf.iat[1,0])

Empty dataframe for MSD was created
      Time(s) d_x track 1 d_y track 1 d_x track 3 d_y track 3 d_x track 4  \
0           0         NaN         NaN         NaN         NaN         NaN   
1           1         NaN         NaN         NaN         NaN         NaN   
2           2         NaN         NaN         NaN         NaN         NaN   
3           3         NaN         NaN         NaN         NaN         NaN   
4           4         NaN         NaN         NaN         NaN         NaN   
...       ...         ...         ...         ...         ...         ...   
6784     6784         NaN         NaN         NaN         NaN         NaN   
6785     6785         NaN         NaN         NaN         NaN         NaN   
6786     6786         NaN         NaN         NaN         NaN         NaN   
6787     6787         NaN         NaN         NaN         NaN         NaN   
6788     6788         NaN         NaN         NaN         NaN         NaN   

     d_y track 4 d_x track 5 d_y track 

# Calculate displacement from origin

In [42]:
mr = tracksDf.shape[0];
mc = tracksDf.shape[1];

for i in range (0, mc):
    if (i%3 == 1 or i%3 == 2):
        trackCoordinates = tracksDf.iloc[:,i].dropna()
        initPosition = trackCoordinates[0] #initial coordinate of particle
        dR = pixSize*(trackCoordinates - initPosition);#calculate displacement
        colR = i//3*2 + i%3 # column for displacement
        dfdR.iloc[0:len(dR),colR] = dR;
print(dfdR)

      Time(s) d_x track 1 d_y track 1 d_x track 3 d_y track 3  d_x track 4  \
0           0         0.0         0.0         0.0         0.0     0.000000   
1           1    0.030836     0.06044    0.679756       0.333     0.766604   
2           2    0.005489    0.263261    0.394436   -0.238334     0.800223   
3           3      0.1507    0.499537   -0.029896   -0.132644     0.788810   
4           4    0.039443    0.143572    0.574584    0.230415     0.651035   
...       ...         ...         ...         ...         ...          ...   
6784     6784         NaN         NaN         NaN         NaN     0.059675   
6785     6785         NaN         NaN         NaN         NaN    -0.185763   
6786     6786         NaN         NaN         NaN         NaN     0.495314   
6787     6787         NaN         NaN         NaN         NaN     0.763799   
6788     6788         NaN         NaN         NaN         NaN     0.640651   

      d_y track 4 d_x track 5 d_y track 5 d_x track 6  ... d_x 

## Save (optional)

In [27]:
root = tk.Tk() # open tkinter gui
path_save = tkinter.filedialog.asksaveasfilename()
root.destroy() # close tkinter
dfdR.to_csv(path_save+'.csv',index=False)